In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import datetime
import pandas_datareader.data as web

In [2]:
start = datetime.datetime(2019, 9, 27)
end = datetime.datetime(2020, 10, 8)
btc = web.DataReader('BTC-USD','yahoo', start, end)

In [3]:
btc = btc[~btc.index.duplicated(keep='first')]

In [4]:
btc = btc.drop(columns=['High','Low','Open','Close','Volume'])

In [5]:
btc.columns = ['Price']

In [6]:
btc = btc.reset_index()

In [7]:
r = pd.date_range(start=btc.Date.min(), end=btc.Date.max())
btc = btc.set_index('Date').reindex(r).rename_axis('Date').reset_index()

In [8]:
btc['Date'] = pd.to_datetime(btc['Date'])
btc.set_index('Date',inplace=True)

In [9]:
btc = btc.fillna(method='ffill')
btc = btc.fillna(method='bfill')

In [10]:
btc = btc.loc['2019-09-27':'2020-10-08']

In [11]:
start = datetime.datetime(2019, 9, 27)
end = datetime.datetime(2020, 10, 8)
sc = web.DataReader('SC-USD','yahoo', start, end)

In [12]:
sc = sc.loc['2019-09-27':'2020-10-08']

In [13]:
sc = sc[~sc.index.duplicated(keep='first')]

In [14]:
sc = sc.drop(columns=['High','Low','Open','Close'])

In [15]:
sc.columns = ['Price','Volume']

In [16]:
sc = sc.reset_index()

In [17]:
r = pd.date_range(start=sc.Date.min(), end=sc.Date.max())
sc = sc.set_index('Date').reindex(r).rename_axis('Date').reset_index()

In [18]:
sc['Date'] = pd.to_datetime(sc['Date'])
sc.set_index('Date',inplace=True)

In [19]:
sc = sc.fillna(method='ffill')
sc = sc.fillna(method='bfill')

In [20]:
import requests
from pandas.io.json import json_normalize

In [21]:
url = 'https://siastats.info/dbs/miningdb.json'
sc_hr = pd.read_json(url)

In [22]:
sc_hr['date'] = pd.date_range(start='6/6/2015', periods=len(sc_hr), freq='D')

In [23]:
sc_hr = sc_hr.drop(columns=['time','difficulty','blocktime'])

In [24]:
sc_hr['date'] = pd.to_datetime(sc_hr['date'])
sc_hr.set_index('date',inplace=True)

In [25]:
sc_hr = sc_hr.loc['2019-09-27':'2020-10-08']

In [26]:
url1 = 'https://siastats.info/dbs/activecontracts.json'
sc_contr = pd.read_json(url1)

In [27]:
sc_contr['date'] = pd.date_range(start='6/6/2015', periods=len(sc_contr), freq='D')

In [28]:
sc_contr = sc_contr.drop(columns=['time','activecontractsize','activecontractcost'])

In [29]:
sc_contr['date'] = pd.to_datetime(sc_contr['date'])
sc_contr.set_index('date',inplace=True)

In [30]:
sc_contr = sc_contr.loc['2019-09-27':'2020-10-08']

In [31]:
url2 = 'https://siastats.info/dbs/storage.json'
sc_stor = pd.read_json(url2)

In [32]:
sc_stor['date'] = pd.date_range(start='9/11/2017', periods=len(sc_stor), freq='D')

In [33]:
sc_stor = sc_stor.drop(columns=['total'])

In [34]:
sc_stor['date'] = pd.to_datetime(sc_stor['date'])
sc_stor.set_index('date',inplace=True)

In [35]:
sc_stor = sc_stor.loc['2019-09-27':'2020-10-08']

In [36]:
url3 = 'https://siastats.info/dbs/transactionsdb.json'
sc_tx = pd.read_json(url3)

In [37]:
sc_tx['date'] = pd.date_range(start='6/6/2015', periods=len(sc_tx), freq='D')

In [38]:
sc_tx = sc_tx.drop(columns=['time','totaltx'])

In [39]:
sc_tx['date'] = pd.to_datetime(sc_tx['date'])
sc_tx.set_index('date',inplace=True)

In [40]:
sc_tx = sc_tx.loc['2019-09-27':'2020-10-08']

In [41]:
frames = [btc,sc,sc_hr,sc_contr,sc_stor,sc_tx]

In [42]:
sc_multy = pd.concat(frames,sort=False,axis=1)

In [43]:
sc_multy.columns = ['BTC_Price','SC_Volume','SC_Price','Hashrate','Contracts','Used_Storage','Daily_Tx']

In [44]:
sc_multy = sc_multy.fillna(method='ffill')
sc_multy = sc_multy.fillna(method='bfill')

In [45]:
sc_multy = sc_multy.reindex(columns = ['BTC_Price','SC_Volume','Hashrate','Contracts','Used_Storage','Daily_Tx','SC_Price'])

In [46]:
sc_multy.tail()

,BTC_Price,SC_Volume,Hashrate,Contracts,Used_Storage,Daily_Tx,SC_Price
2020-10-04,10804.000977,1624328.0,8426.050178,195124.0,690.84,14210.0,0.002826
2020-10-05,10621.664062,1354025.0,8338.793499,192975.0,690.84,19466.0,0.002637
2020-10-06,10679.136719,1005155.0,8632.116299,191107.0,690.84,31968.0,0.002692
2020-10-07,10679.136719,1005155.0,8632.116299,191107.0,690.84,31968.0,0.002692
2020-10-08,10647.002930,1697586.0,8632.116299,191107.0,690.84,31968.0,0.002709


In [47]:
sc_multy = sc_multy.drop(columns='BTC_Price')

In [49]:
x = sc_multy.iloc[:,:-1].values
y = sc_multy.iloc[:,-1].values

In [50]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [51]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=10, random_state = 0)
regressor.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [52]:
y_pred = regressor.predict(x_test)
np.set_printoptions(precision=4)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0.0032 0.0028]
 [0.0025 0.0023]
 [0.0032 0.0032]
 [0.0032 0.0035]
 [0.0024 0.0023]
 [0.0019 0.0017]
 [0.0044 0.005 ]
 [0.0019 0.0016]
 [0.002  0.0017]
 [0.0019 0.002 ]
 [0.0025 0.0024]
 [0.0032 0.0029]
 [0.0025 0.0026]
 [0.0023 0.0022]
 [0.0021 0.0021]
 [0.0019 0.0015]
 [0.0032 0.0031]
 [0.0032 0.0034]
 [0.0019 0.0018]
 [0.0015 0.0011]
 [0.0026 0.0028]
 [0.0022 0.002 ]
 [0.0026 0.0027]
 [0.0033 0.0036]
 [0.0015 0.0013]
 [0.0015 0.0013]
 [0.0034 0.0038]
 [0.0021 0.0019]
 [0.0035 0.0034]
 [0.0015 0.0014]
 [0.003  0.0026]
 [0.0019 0.0017]
 [0.0019 0.0016]
 [0.0032 0.0031]
 [0.0019 0.0021]
 [0.0019 0.0012]
 [0.0023 0.0022]
 [0.0025 0.0031]
 [0.0015 0.0015]
 [0.002  0.0017]
 [0.0019 0.002 ]
 [0.0019 0.0019]
 [0.0019 0.0015]
 [0.0016 0.0013]
 [0.0032 0.0032]
 [0.0015 0.0013]
 [0.0032 0.003 ]
 [0.0019 0.0023]
 [0.0019 0.0019]
 [0.0015 0.0016]
 [0.0019 0.0016]
 [0.0032 0.0031]
 [0.0015 0.0014]
 [0.0019 0.0017]
 [0.0027 0.0028]
 [0.0032 0.0029]
 [0.0035 0.0045]
 [0.0031 0.0029]
 [0.0028 0.003

In [53]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.8850854794786391